In [1]:
import torch

In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else (torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu"))
device

device(type='mps')

In [76]:
# Representation Neural Network
from torch.nn import Module, ModuleList
from torch.nn import Conv2d, Linear
from torch.nn import ReLU, BatchNorm2d

class ConvolutionalBlock(Module):

    def __init__(self, in_channels, n_channels = 256, kernel_size = (3,3), stride=(1,1), padding=(1,1)):
        super(ConvolutionalBlock, self).__init__()
        self.convo = Conv2d(in_channels=in_channels, out_channels=n_channels, kernel_size=kernel_size, padding=padding, stride=stride)
        self.norm = BatchNorm2d(num_features=n_channels)
        self.act = ReLU()

    def forward(self, x):
        return self.act(self.norm(self.convo(x)))
    
x = torch.randn((1,1,3,4), device=device)
convo_block = ConvolutionalBlock(1).to(device=device)

print(x.shape, convo_block(x).shape)

torch.Size([1, 1, 3, 4]) torch.Size([1, 256, 3, 4])


In [66]:
class ResidualBlock(Module):

    def __init__(self, in_channels, n_channels = 256, kernel_size = (3,3), stride=(1,1)):
        super(ResidualBlock, self).__init__()
        self.convo1 = ConvolutionalBlock(in_channels=in_channels, n_channels=n_channels, kernel_size=kernel_size, stride=stride)
        self.convo2 = Conv2d(in_channels=n_channels, out_channels=n_channels, kernel_size=kernel_size, padding=(1,1), stride=stride)
        self.norm = BatchNorm2d(num_features=n_channels)
        self.act = ReLU()

    def forward(self, x):
        skip_conn = x
        x = self.convo1(x)
        x = self.norm(self.convo2(x))
        x = x + skip_conn
        x = self.act(x)
        return x

x = torch.randn((1,1,3,4), device=device)
convo_block = ResidualBlock(1).to(device=device)

print(x.shape, convo_block(x).shape)

torch.Size([1, 1, 3, 4]) torch.Size([1, 256, 3, 4])


In [67]:
class GenericResidualNetwork(Module):

    def __init__(self, in_channels, n_channels = 256, n_layers=10, kernel_size = (3,3)):
        super(GenericResidualNetwork, self).__init__()

        # For now, simple, no downscale
        self.input_layer = ResidualBlock(in_channels=in_channels, n_channels=n_channels, kernel_size=kernel_size)

        self.residuals = ModuleList([ResidualBlock(in_channels=n_channels, n_channels=n_channels, kernel_size=kernel_size) for _ in range(n_layers-1)])

    def forward(self, x):

        x = self.input_layer(x)
        for res_layer in self.residuals:
            x = res_layer(x)
        return x
    
x = torch.randn((1,1,6,7), device=device)
convo_block = GenericResidualNetwork(1).to(device=device)

x = convo_block(x)
x.shape

torch.Size([1, 256, 6, 7])

In [ ]:
class PolicyPredictor(Module):

    def __init__(self, in_channels, num_conv = 2 ):
        
        self.conv1 = Conv2d() 

In [70]:
class ContinousValuePredictor(Module):

    def __init__(self, in_channels, board_total_slots, n_outputs, n_convs = 2):
        super(ContinousValuePredictor, self).__init__()

        self.convos = ModuleList()
        
        for _ in range(n_convs):
            out_channels = in_channels // 2
            convo = ConvolutionalBlock(in_channels=in_channels, n_channels=out_channels, kernel_size=(1,1), padding=(0,0))
            self.convos.append(convo)
            in_channels = out_channels

        self.output_size = board_total_slots * out_channels
        self.linear = Linear(self.output_size, n_outputs)

    def forward(self, x):

        for conv in self.convos:
            x = conv(x)

        x = x.view(-1, self.output_size)
        return self.linear(x)       

x = torch.randn((1,256,6,7), device=device)
convo_block = ContinousValuePredictor(256, 42, 601).to(device=device)

x = convo_block(x)
x.shape 

torch.Size([1, 601])

In [73]:
class DynamicsNetwork(Module):

    def __init__(self, in_channels, board_total_slots, n_convs = 2, n_channels = 256, n_residual_layers=10, kernel_size = (3,3)):
        super(DynamicsNetwork, self).__init__()

        self.first_net = GenericResidualNetwork(in_channels=in_channels, n_channels=n_channels, n_layers=n_residual_layers, kernel_size=kernel_size)
        self.reward_predictor = ContinousValuePredictor(in_channels=n_channels, board_total_slots=board_total_slots, n_outputs=601, n_convs=n_convs)

    def forward(self, x):

        return self.reward_predictor(self.first_net(x))
    

x = torch.randn((1,256,6,7), device=device)
convo_block = DynamicsNetwork(256, 42).to(device=device)

x = convo_block(x)
x.shape 

torch.Size([1, 601])

In [84]:
class PredictionNetwork(Module):

    def __init__(self, in_channels, board_total_slots, action_space_size, n_convs = 2, n_channels = 256, n_residual_layers=10, kernel_size = (3,3)):
        super(PredictionNetwork, self).__init__()

        self.first_net = GenericResidualNetwork(in_channels=in_channels, n_channels=n_channels, n_layers=n_residual_layers, kernel_size=kernel_size)
        self.value_predictor = ContinousValuePredictor(in_channels=n_channels, board_total_slots=board_total_slots, n_outputs=601, n_convs=n_convs)
        self.policy_predictor = ContinousValuePredictor(in_channels=n_channels, board_total_slots=board_total_slots, n_outputs=action_space_size, n_convs=n_convs)

    def forward(self, x):

        x = self.first_net(x)
        pp = self.policy_predictor(x)
        return self.value_predictor(x), torch.nn.functional.softmax(pp, dim=1)
    

x = torch.randn((1,256,6,7), device=device)
convo_block = PredictionNetwork(256, 42, 7).to(device=device)

x = convo_block(x)
x

(tensor([[-0.1555, -0.8329,  0.1847, -0.1942,  0.3341,  0.2840,  0.0784,  0.1169,
          -0.5542, -0.0793, -0.9286, -0.1915, -1.0725,  0.6287,  0.0216, -0.4392,
           0.1499, -0.1900, -0.3710, -0.9801,  0.3121, -0.1348, -0.5337,  0.2209,
          -0.2979, -0.0740, -0.5907,  0.3962,  0.5812, -0.2135, -0.1696,  0.0710,
          -0.8737, -0.6410,  0.2240,  0.5494, -0.1721,  0.2564, -0.1549,  0.1967,
           0.2573, -0.6970,  0.1928,  0.7517, -0.2378,  0.6223,  0.9036,  0.7385,
           0.2385, -0.3269,  0.3727,  0.2111,  0.5860,  0.5001, -0.2817, -0.2401,
          -0.2091,  0.0435,  0.4508, -0.4706, -0.1902,  0.1987,  0.5536,  0.0138,
           0.3129,  0.3344,  0.4260,  0.6085, -0.7781, -0.4287, -0.7040, -0.4321,
          -0.5182,  0.0289,  0.0201,  0.0366, -0.2440,  0.0718,  0.4529, -0.7286,
          -0.0469, -0.2206,  0.4698,  0.3199, -0.4301,  0.3965,  0.4098,  0.0192,
           0.3331,  0.5308,  0.5195, -0.4866, -0.2346,  0.2618, -0.2199, -0.3240,
          -0.044